In [1]:
import numpy as np
import pickle
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [20]:
import keras
import numpy as np
from keras.layers import Dense, Input, Dropout, CuDNNLSTM, LSTM, Activation,BatchNormalization,Lambda,concatenate,Concatenate,Multiply,Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import Model,load_model
from keras import regularizers
from keras import backend as K
max_len =200
np.random.seed(1)

In [21]:
from keras.preprocessing import text,sequence

In [22]:
cre_vector  = KeyedVectors.load("./embed_artifact/creative_sg_embed_s128_wv_s0oc73kx")
ad_vector   = KeyedVectors.load("./embed_artifact/ad_sg_embed_s128_wv_gdyyb1qc")
adv_vector  = KeyedVectors.load("./embed_artifact/advertiser_sg_embed_s128_wv_t70t5_xl")
ind_vector  = KeyedVectors.load("./embed_artifact/industry_sg_embed_s64_wv_bg7iidgr")
prod_vector = KeyedVectors.load("./embed_artifact/product_sg_embed_s128_wv_dyplgj8j")

In [5]:
with open("./input_artifact/train_creative_id_seq.pkl","rb+") as f:
    train_cre_id_v = pickle.load(f)

In [6]:
with open("./input_artifact/train_ad_id_seq.pkl","rb+") as f:
    train_ad_id_v = pickle.load(f)

In [7]:
with open("./input_artifact/train_advertiser_id_seq.pkl","rb+") as f:
    train_adv_id_v = pickle.load(f)

In [8]:
with open("./input_artifact/train_industry_id_seq.pkl","rb+") as f:
    train_ind_id_v = pickle.load(f)

In [9]:
with open("./input_artifact/train_product_id_seq.pkl","rb+") as f:
    train_prod_id_v = pickle.load(f)

In [10]:
train_age    = np.load("./input_artifact/train_age.npy")
train_gender = np.load("./input_artifact/train_gender.npy")

In [11]:
#def get_label(label):
#    length = label.max()+1
#    train_label = []
#    for i in label:
#        tmp_label = [0]*length
#        tmp_label[i] = 1
#        train_label.append(tmp_label)
#    return np.array(train_label)
#age_label = get_label(train_age)
#gender_label = get_label(train_gender)

In [12]:
def txt_to_idx(txt,wv):
    
    word_list = wv.index2word
    
    words_dict={}
    for index,num in enumerate(word_list):
        words_dict[num]=index
    
    txt_idx = []
    sentence_idx = []
    for sentence in txt:
        for i in range(len(sentence)):
            sentence_idx.append(words_dict[sentence[i]])
        txt_idx.append(sentence_idx)
    return txt_idx

train_cre_idx = txt_to_idx(train_cre_id_v,cre_vector)
train_ad_idx  = txt_to_idx(train_ad_id_v,ad_vector) 
train_adv_idx = txt_to_idx(train_adv_id_v,adv_vector)
train_ind_idx = txt_to_idx(train_ind_id_v,ind_vector) 
train_prod_idx= txt_to_idx(train_prod_id_v,prod_vector)

In [13]:
train_cre_idx = sequence.pad_sequences(train_cre_idx,maxlen = max_len)
train_ad_idx  = sequence.pad_sequences(train_ad_idx, maxlen = max_len)
train_adv_idx = sequence.pad_sequences(train_adv_idx,maxlen = max_len)
train_ind_idx = sequence.pad_sequences(train_ind_idx,maxlen = max_len)
train_prod_idx= sequence.pad_sequences(train_prod_idx,maxlen= max_len)

In [25]:
def pretrained_embedding_layer(target_matrix):    
    vocab_len = target_matrix.shape[0]
    emb_dim = target_matrix.shape[1]
    emb_matrix = target_matrix
    embedding_layer = Embedding(vocab_len,emb_dim,trainable = False,mask_zero = True)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    return embedding_layer

In [26]:
def lstm_model(max_len = 100):
    #ad输入
    ad_txt = Input(shape=(max_len,), dtype='int32')
    embedding_layer_ad =  pretrained_embedding_layer(ad_vector.vectors)
    ad_input = embedding_layer_ad(ad_txt)
    #creative输入
    creative_txt = Input(shape=(max_len,), dtype='int32')
    embedding_layer_creative =  pretrained_embedding_layer(cre_vector.vectors)
    creative_input = embedding_layer_creative(creative_txt)
    #advertiser输入
    adv_txt = Input(shape=(max_len,), dtype='int32')
    embedding_layer_adv =  pretrained_embedding_layer(adv_vector.vectors)
    adv_input = embedding_layer_adv(adv_txt)
    #industry输入
    ind_txt = Input(shape=(max_len,), dtype='int32')
    embedding_layer_ind =  pretrained_embedding_layer(ind_vector.vectors)
    ind_input = embedding_layer_ind(ind_txt)
    #product输入
    prod_txt = Input(shape=(max_len,), dtype='int32')
    embedding_layer_prod =  pretrained_embedding_layer(prod_vector.vectors)
    prod_input = embedding_layer_prod(prod_txt)
    #天数输入
    #day_idx = Input(shape=(max_len,), dtype='int32')
    #Day = Dense(128, activation='softmax')(day_idx)
    
    #LSTM层
    LSTM_ad = LSTM(128,return_sequences=1)(ad_input)
    LSTM_creative = LSTM(128,return_sequences=1)(creative_input)
    LSTM_adv = LSTM(128,return_sequences=1)(adv_input)
    LSTM_ind = LSTM(64,return_sequences=1)(ind_input)
    LSTM_prod = LSTM(128,return_sequences=1)(prod_input)
    
    #全连接神经网络层
    x = concatenate([LSTM_ad,LSTM_creative,LSTM_adv,LSTM_ind,LSTM_prod],axis = -1)
    
    #attention
    Q_age = Dense(128*4+64,use_bias = None,activation = "softmax",name = "age_attention")(x)
    x_age = Multiply()([x, Q_age])
    
    Q_gender = Dense(128*4+64,use_bias = None,activation = "softmax",name = "gender_attention")(x)
    x_gender = Multiply()([x, Q_gender])
    
    x_age = CuDNNLSTM(128,return_sequences=1)(x_age)
    
    x_gender = CuDNNLSTM(128,return_sequences=1)(x_gender)
    
    x_age_mx = Lambda(K.max,arguments={"axis":1})(x_age)
    x_age_av = Lambda(K.mean,arguments={"axis":1})(x_age)
    
    x_gender_mx = Lambda(K.max,arguments={"axis":1})(x_gender)
    x_gender_av = Lambda(K.mean,arguments={"axis":1})(x_gender)
    
    x_age = Concatenate()([x_age_mx,x_age_av])
    x_gender = Concatenate()([x_gender_mx,x_gender_av])
    
    x_age = Dropout(0.2)(x_age)
    x_age = Dense(512, activation='relu')(x_age)
    x_age = Dropout(0.2)(x_age)
    x_age = Dense(10, activation='softmax')(x_age)
    
    x_gender = Dropout(0.2)(x_age)
    x_gender = Dense(512, activation='relu')(x_gender)
    x_gender = Dropout(0.2)(x_age)
    x_gender = Dense(1, activation='sigmoid')(x_gender)
    
    model = Model(inputs=[ad_txt, 
                          creative_txt,
                          adv_txt,
                          ind_txt,
                          prod_txt
                          ], 
                  outputs=[x_age,
                           x_gender
                          ], 
                  )
    return model

In [27]:
model = lstm_model(max_len)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 200)]        0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, 200)]        0                                            
__________________________________________________________________________________________________
input_13 (InputLayer)           [(None, 200)]        0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, 200)]        0                                            
____________________________________________________________________________________________

In [17]:
del cre_vector,ad_vector,adv_vector,ind_vector,prod_vector
import gc
gc.collect()

20

In [28]:
model.compile(optimizer="adam", loss=['sparse_categorical_crossentropy','binary_crossentropy'],
              loss_weights=[0.5, 0.5],
              metrics = ['accuracy']
              )

In [29]:
model.fit([train_ad_idx,train_cre_idx,train_adv_idx,train_ind_idx,train_prod_idx],
          [train_age,train_gender],
          #verbose=2,
          epochs=10,
          batch_size=64
          )

Train on 900000 samples


InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNNV2' used by node cu_dnnlstm_1/CudnnRNNV2 (defined at C:\Users\86188\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py:1748) with these attrs: [seed=0, dropout=0, T=DT_FLOAT, input_mode="linear_input", direction="unidirectional", rnn_mode="lstm", seed2=0, is_training=true]
Registered devices: [CPU]
Registered kernels:
  device='GPU'; T in [DT_HALF]
  device='GPU'; T in [DT_FLOAT]
  device='GPU'; T in [DT_DOUBLE]

	 [[cu_dnnlstm_1/CudnnRNNV2]]

In [29]:
model.save("model_lstm_5input_128dim_100maxlen.model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model_lstm_5input_128dim_100maxlen.model/assets


In [30]:
from keras.models import load_model
model1 = load_model("model_lstm_5input_128dim_100maxlen.model")

In [32]:
model1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 100)]        0                                            
______________________________________________________________________________________________